# Compare performance of TMCMC vs others

In [51]:
import numpy as np
import multiprocess as mp
import itertools
import time

from UQpy.sampling import MetropolisHastings
from UQpy.distributions import Normal, Uniform, MultivariateNormal

from parallel_tempMCMC import SequentialTemperingMCMCpar
from temper_benchmark import scenario_logp, scenario_logC, precise_risk, log_pdf_intermediate
from highway_risk_temper import get_damage_state

## Constants shared by different evaluation methods

In [6]:
# set random_state for repeatability
random_state = 1

# parallel computing
n_jobs = 1

# small number of numerical stability
eps = 1e-6

# bridge number and their reliability limits
n_br, min_beta, max_beta = 30, 0, 3+eps

# generate beta array for all bridge links
beta_array = Uniform(loc=min_beta, scale=max_beta-min_beta,).rvs(
    nsamples=n_br, random_state=1)
beta_array = beta_array.flatten()
beta_array.sort()
pf_array = Normal().cdf(-beta_array)

Change `cost_type` between `'add'` and `'swan'` to compare additive cost and grey-swan cost, respectively. Define `cost_array` and `cost_base` based on `cost_type`.

In [7]:
cost_type = 'add'

if cost_type == 'add':
    cost_array = np.arange(1.0, 1.0+n_br, 1)
    cost_base = 0.0
elif cost_type == 'swan':
    cost_array = np.power(10**beta_array, np.linspace(0.0, 1.0, n_br))
    cost_array[:-3] = 1e-3
    cost_base = 1.1    # when cost_type='swan', having a cost_base larger than 1 improves resampling process
else:
    raise RuntimeError("Unknown cost_type: must be 'add' or 'swan'")

**The precise risk for this problem can be determined as follows**

In [8]:
risk0 = precise_risk(pf_array, cost_array, cost_type=cost_type, cost_base=cost_base)
print(f'precise risk = {risk0}')

precise risk = 39.448804452293125


## TMCMC Results

### Set up TMCMC

In [9]:
# TMCMC constants
n_chains, resample_pct, n_smp = 100, 10, 5000
n_burn, n_jump = 1000, 10
covar = 0.5**2

In some combinations of `resample_pct`, `n_smp`, and `n_chains`, the true resampling percentage may be different from the specified `resample_pct`.

The following cell is optional and provides the true resampling percentage used in TMCMC.

In [10]:
n_samples_per_chain = int(np.floor(((1 - resample_pct/100) * n_smp) / n_chains))
n_resamples = int(n_smp - (n_samples_per_chain * n_chains))

# no. of chains cannot be higher than the no. of resamples points
if n_resamples < n_chains:
    # reduce no of chains
    n_chains = n_resamples

print(f'True resampling pct is {n_resamples/n_smp} (nominal: {resample_pct/100})')

True resampling pct is 0.1 (nominal: 0.1)


### Conduct TMCMC

The following parameters seem to have a significant impact on the sampling quality:

* `resample_pct` that controls how many samples are reused between stages. 10% from the literature seems perform well here.

* Parameters of `mcmc_sampler`, including `n_chains` (number of Markov chains with different heads), `n_burn` (number of burn-in samples), and `n_jump` (number of samples to skip along a Markov chain)

Note that the way `random_state` is used is different between serial run and parrallel run. Therefore, the results are different even though the same `random_state` is used.

Also, when using parallel computing, `damage_db` will not be populated.

In [11]:
# damage database to avoid redundant computation
damage_db = dict()

# prior distribution of hidden variables mapped to asset state
prior = MultivariateNormal(mean=[0.0]*n_br, cov=1.0)

# mcmc_sampler is the MCMC used to generate new samples (after resampling)
mcmc_sampler = MetropolisHastings(
    dimension=n_br, n_chains=n_chains,
    proposal=MultivariateNormal(mean=[0.0]*n_br, cov=covar),
    burn_length=n_burn, jump=n_jump,
)

# provide explicit prior log_pdf (due to multiprocess cannot pickle objects)
prior_log_pdf = lambda x: prior.log_pdf(x)

# intermediate likelihood function
use_log_pdf = lambda x, b: log_pdf_intermediate(
    x, b, beta_array=beta_array, cost_array=cost_array,
    cost_base=cost_base, damage_db=damage_db, epsilon=eps)

Create TMCMC sampler and obtain the samples.

In [12]:
sampler = SequentialTemperingMCMCpar(
    log_pdf_intermediate=use_log_pdf,
    distribution_reference=prior,
    save_intermediate_samples=True,
    percentage_resampling=resample_pct,
    sampler=mcmc_sampler,
    weight_cov_threshold=0.2,
    random_state=random_state,
    nsamples=n_smp,
)
time0 = time.time()
if n_jobs == 1:
    sampler.run(nsamples=n_smp)
else:
    sampler.parallel_run(nsamples=n_smp, n_jobs=n_jobs,
                         log_pdf_intermediate=use_log_pdf,
                         prior_log_pdf=prior_log_pdf)
time1 = time.time()

# get samples
samples = sampler.samples

# retrieve damage condition and unique damage condition from last-stage samples
damage_condition = get_damage_state(samples, beta_array).astype(int)
unique_condition, counts = np.unique(damage_condition, axis=0, return_counts=True)

**Report sampling results:**

* `evidence`: estimated risk plus `cost_base`

* `max_analysis`: number of unique cost evaluations during sampling

* `max_smp`: number of samples generated in all stages

The latter two (`max_analysis` and `max_smp`), together with running time (`time1-time0`), control the termination of crude Monte Carlo simulation.

In [13]:
# get the estimated evidence
evidence = sampler.evidence

total_analysis = len(list(damage_db))
all_smp = np.vstack(sampler.intermediate_samples)
total_smp = all_smp.shape[0]*n_jump
total_time = (time1-time0)*n_jobs

print(f'Completed in {total_time} sec')
print(f'number of chains: {n_chains}')
print(f'number of stages: {len(sampler.intermediate_samples)}')
print(f'samples per stage: {n_smp}')
print(f'number of all samples: {total_smp}')
print(f'number of net analysis: {total_analysis}')
print(f'evidence estimated: {evidence[0]}')
print(f'risk estimated (TMCMC): {evidence[0]-cost_base}')

Completed in 10.836361646652222 sec
number of chains: 100
number of stages: 4
samples per stage: 5000
number of all samples: 200000
number of net analysis: 0
evidence estimated: 38.78292577743867
risk estimated (TMCMC): 38.78292577743867


## MC Results

### Set up MC

To achieve fair comparison, MC is terminated based on computational costs (`max_analysis`. `max_smp`, and runtime) from TMCMC. Therefore, TMCMC must be run first before runnning subsequent cells.

Alternatively, one can uncomment lines in the following cell to run MC directly.

In [22]:
# max_analysis = 10000
# max_smp = 100000
# max_time = np.inf

max_analysis = total_analysis if total_analysis>0 else 2**n_br
max_smp = total_smp
max_time = total_time

### Conduct MC

Get MC samples

In [23]:
condition_rv = MultivariateNormal(mean=[0.0]*n_br, cov=1.0)
condition_rvs_max = condition_rv.rvs(nsamples=max_smp, random_state=random_state)

damage_db_MC = dict()
C_smp_list = []
time0 = time.time()
for i, condition_rvs in enumerate(condition_rvs_max):

    condition_rvs = condition_rvs.reshape((-1,1))

    logC = scenario_logC(
        condition_rvs, beta_array=beta_array,
        cost_array=cost_array,
        from_condition=False,
        cost_type=cost_type,
        cost_base=cost_base,
        damage_db=damage_db_MC,
        epsilon=eps,
    )

    C_smp = np.exp(logC).flatten()[0]
    C_smp_list.append(C_smp)
    
    time1 = time.time()

    # check if terminate before max_samp
    n_analysis = len(list(damage_db_MC))
    if (n_analysis >= max_analysis) or (time1-time0>max_time):
        break

C_smp_array = np.array(C_smp_list)

**Report results**

In [24]:
R_net = np.mean(C_smp_array)

print(f'Completed in {time1-time0} sec')
print(f'number of samples: {C_smp_array.shape[0]} vs {max_smp}')
print(f'number of net analysis: {len(list(damage_db_MC))} vs. {max_analysis}')
print(f'risk estimated: {R_net}')

Completed in 10.836597919464111 sec
number of samples: 26028 vs 200000
number of net analysis: 0 vs. 1073741824
risk estimated: 39.72521999492854


## Risk-bound Results

### Set up risk-bound

Similary, the following cells require running TMCMC first to achieve fair comparison.

Also, since it requires a non-empty `damage_db`, TMCMC should be conducted using the series version.

In [42]:
# if damage_db is empty, assume all samples require a new cost evaluation
total_scenario = total_analysis if total_analysis>0 else max_smp

# n_fail = 2
# total_scenario = np.sum([comb(n_br, j) for j in range(1, n_fail+1)])

### Conduct risk-bound estimation

In [57]:
sorting_indx = cost_array.argsort()[::-1]
comb_condition = []
nsc = 0
for failed in range(0, n_br+1):
    for indx in itertools.combinations(sorting_indx, failed):
        key = np.zeros(n_br, dtype=bool)
        key[(indx,)] = True
        comb_condition.append(key)
        nsc += 1
        if nsc >= total_scenario:
            break
    if nsc >= total_scenario:
        break

comb_condition = np.array(comb_condition)
logp_comb = scenario_logp(comb_condition, beta_array, from_condition=True)
logC_comb = scenario_logC(comb_condition, beta_array, cost_array,
                          from_condition=True, cost_type=cost_type,
                          cost_base=cost_base, damage_db=None, epsilon=eps)

all_fail = np.ones((1, n_br), dtype=bool)
logC_max = scenario_logC(all_fail, beta_array, cost_array,
                         from_condition=True, cost_type=cost_type,
                         cost_base=cost_base, damage_db=None, epsilon=eps)[0]

logC_min = np.min(cost_array.min())


**Report results.**

In [62]:
# estimate risk
logpC_comb = logp_comb + logC_comb
risk_bound0 = np.sum(np.exp(logpC_comb))

# remaining probs
p_remain = np.maximum(0, 1-np.exp(logp_comb).sum())
risk_bound1 = risk_bound0 + p_remain*np.exp(logC_min)
risk_bound2 = risk_bound0 + p_remain*np.exp(logC_max)

print(f'estimation (no remaining) = {risk_bound0}')
print(f'estimation (w/ remaining) lower = {risk_bound1}')
print(f'estimation (w/ remaining) upper = {risk_bound2}')

estimation (no remaining) = 16.671498174697582
estimation (w/ remaining) lower = 17.795304964223387
estimation (w/ remaining) upper = 208.91431596579582
